In [2]:
pip install hyperopt

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
    --------------------------------------- 0.0/1.6 MB 435.7 kB/s eta 0:00:04
   - -------------------------------------- 0.1/1.6 MB 656.4 kB/s eta 0:00:03
   --- ------------------------------------ 0.1/1.6 MB 983.0 kB/s eta 0:00:02
   ------ --------------------------------- 0.3/1.6 MB 1.5 MB/s eta 0:00:01
   -------- ------------------------------- 0.3/1.6 MB 1.5 MB/s eta 0:00:01
   ------------- -------------------------- 0.5/1.6 MB 1.9 MB/s eta 0:00:01
   ----------------- ---------------------- 0.7/1.6 MB 2.2 MB/s eta 0:00:01
   ---------------------- ----------------- 0.9/1.6 MB 2.5 MB/s eta 0:00:01
   -------------------------- ------------- 1.1/1.6 MB 2.6 MB/s eta 0:00:01
   ------------------------------- -------- 1.2/1.6 MB 2.7 MB/s eta 0:00:01
   ----------------------------------- ---- 1.4/1.6 MB 2.8 MB/s eta 0:00:01
   --------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [8]:
pip install joblib~=1.1.0

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scikit-learn 1.4.2 requires joblib>=1.2.0, but you have joblib 1.1.1 which is incompatible.
ydata-profiling 0.0.dev0 requires visions[type_image_path]<0.7.7,>=0.7.5, but you have visions 0.7.4 which is incompatible.
ydata-profiling 0.0.dev0 requires wordcloud>=1.9.1, but you have wordcloud 0.0.0 which is incompatible.



  Using cached joblib-1.1.1-py2.py3-none-any.whl.metadata (5.2 kB)
Using cached joblib-1.1.1-py2.py3-none-any.whl (309 kB)
  Attempting uninstall: joblib
    Found existing installation: joblib 1.4.2
    Uninstalling joblib-1.4.2:
      Successfully uninstalled joblib-1.4.2


In [11]:
pip install statsforecast

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pytorch-lightning

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/815.2 kB ? eta -:--:--
   - ------------------------------------- 41.0/815.2 kB 960.0 kB/s eta 0:00:01
   ----- ---------------------------------- 112.6/815.2 kB 1.3 MB/s eta 0:00:01
   ------------- -------------------------- 276.5/815.2 kB 2.1 MB/s eta 0:00:01
   ---------------- ----------------------- 337.9/815.2 kB 1.9 MB/s eta 0:00:01
   ----------------------- ---------------- 471.0/815.2 kB 2.1 MB/s eta 0:00:01
   ----------------------------- ---------- 604.2/815.2 kB 2.2 MB/s eta 0:00:01
   ------------------------------------ --- 737.3/815.2 kB 2.3 MB/s eta 0:00:01
   ---------------------------------------- 815.2/815.2 kB 2.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/926.4 kB ? eta -:--:--
   ----- ---------------------------------- 122.9/926.4 kB 3.5 MB/s eta 0:00:01
   ------------ --------------------------- 286.7/926.4 kB 3

In [2]:
import os
import pickle
import time
import argparse

import numpy as np
import pandas as pd

from hyperopt import fmin, tpe, hp, Trials, STATUS_OK

from statsforecast import StatsForecast
from statsforecast.models import (
    Naive,
    SeasonalNaive, 
    AutoARIMA 
)

from src.losses.numpy import mae, mse
from src.experiments.utils import hyperopt_tunning

import matplotlib.pyplot as plt

In [3]:
def main(args):

    #----------------------------------------------- Load Data -----------------------------------------------#
    Y_df = pd.read_csv(f'ETTm2.csv')

    X_df = None
    S_df = None

    print('Y_df: ', Y_df.head())
    if args.dataset == 'ETTm2':
        len_val = 11520
        len_test = 11520
    if args.dataset == 'Exchange':
        len_val = 760
        len_test = 1517
    if args.dataset == 'ECL':
        len_val = 2632
        len_test = 5260
    if args.dataset == 'traffic':
        len_val = 1756
        len_test = 3508
    if args.dataset == 'weather':
        len_val = 5270
        len_test = 10539
    if args.dataset == 'ili':
        len_val = 97
        len_test = 193

    #---------------------------------------------- Directories ----------------------------------------------#
    output_dir = f'./results/multivariate/{args.dataset}_{args.horizon}/NHITS/'

    os.makedirs(output_dir, exist_ok = True)
    assert os.path.exists(output_dir), f'Output dir {output_dir} does not exist'
    
    hyperopt_file = output_dir + f'hyperopt_{args.experiment_id}.p'    
    
    n_series = len(Y_df.unique_id.unique())
    Y_df['ds'] = pd.to_datetime(Y_df['ds'])
    # Test set predictions
    model = StatsForecast(df=Y_df[['unique_id', 'ds', 'y']].set_index('unique_id'),
                          models=[AutoARIMA(season_length=args.seasonality)],
                          freq=args.freq, 
                          n_jobs=-1)
    forecasts = model.cross_validation(h=args.horizon, 
                                       n_windows=None, step_size=1,
                                       test_size=len_test)
    forecasts['unique_id'] = forecasts.index
    forecasts = forecasts.reset_index(drop=True)
    
    y_true = forecasts['y'].values
    # y_hat  = forecasts[f'seasonal_naive_season_length-{args.seasonality}'].values
    y_hat  = forecasts[f'SeasonalNaive'].values

    y_true = y_true.reshape(n_series, -1, args.horizon)
    y_hat  = y_hat.reshape(n_series, -1, args.horizon)
    
    #return forecasts, y_true, y_hat
    losses= {'dataset': args.dataset,
             'horizon': args.horizon,
             'mse': mse(y=y_true, y_hat=y_hat),
             'mae': mae(y=y_true, y_hat=y_hat)}
    
    #with open(hyperopt_file, "wb") as f:
    #    pickle.dump(trials, f)    
    
    return losses
    
    #pickle.dump(trials, f)

# source ~/anaconda3/etc/profile.d/conda.sh
# conda activate nixtla
# CUDA_VISIBLE_DEVICES=0 python nhits_multivariate.py --hyperopt_max_evals 10 --experiment_id "eval_train"

In [ ]:
args=pd.Series(dict(experiment_id='seasonal_naive'))
if __name__ == '__main__':
    
    horizons = [96, 192, 336, 720]
    ILI_horizons = [24, 36, 48, 60]
    datasets = ['ETTm2']
    seasonalities = [96, 7, 144, 52, 96, 24]
    freqs = ['15min', 'D', '10min', 'W', '15min', 'H']
    
    #datasets = ['ili']
    #seasonalities = [52]
    #freqs = ['W']
    
    losses_list = []
    for dataset, seasonality, freq in\
            zip(datasets, seasonalities, freqs):
        # Horizon
        if dataset == 'ili':
            horizons_dataset = ILI_horizons
        else:
            horizons_dataset = horizons
        for horizon in horizons_dataset:
            print(50*'-', dataset, 50*'-')
            print(50*'-', horizon, 50*'-')
            start = time.time()
            args.dataset = dataset
            args.horizon = horizon
            args.seasonality = seasonality
            args.freq = freq
            losses = main(args)
            losses_list.append(losses)
            print('Time: ', time.time() - start)


-------------------------------------------------- ETTm2 --------------------------------------------------
-------------------------------------------------- 96 --------------------------------------------------
Y_df:                      ds unique_id         y
0  2016-07-01 00:00:00      HUFL -0.041413
1  2016-07-01 00:00:00      HULL  0.040104
2  2016-07-01 00:00:00      MUFL -0.599211
3  2016-07-01 00:00:00      MULL -0.393536
4  2016-07-01 00:00:00      LUFL  0.695804


C:\Users\bichl\AppData\Roaming\Python\Python312\site-packages\statsforecast\core.py:476: FutureWarning: The `df` argument of the StatsForecast constructor as well as reusing stored dfs from other methods is deprecated and will raise an error in a future version. Please provide the `df` argument to the corresponding method instead, e.g. fit/forecast.
  warnings.warn(
C:\Users\bichl\AppData\Roaming\Python\Python312\site-packages\statsforecast\core.py:628: FutureWarning: Passing unique_id as the index is deprecated. Please provide it as a column instead.
  warnings.warn(
C:\Users\bichl\AppData\Roaming\Python\Python312\site-packages\statsforecast\core.py:476: FutureWarning: The `df` argument of the StatsForecast constructor as well as reusing stored dfs from other methods is deprecated and will raise an error in a future version. Please provide the `df` argument to the corresponding method instead, e.g. fit/forecast.
  warnings.warn(


In [ ]:
eval_df = pd.DataFrame(losses_list)

In [ ]:
eval_df